Base script for comparison existance of output-format in the prompt

In [1]:
# review text

import sys
from pathlib import Path
import pandas as pd

sys.path.append('../../')

import _sample_reviews
import _prompts

from langchain_community.llms import Ollama

# YOUR REVIEW TEXT
review_text = _sample_reviews.sample_11

# YOUR OLLAMA MODEL OF THIS SCRIPT
llm = Ollama(model='mixtral:8x7b-instruct-v0.1-q4_0', temperature=0.4)

In [2]:
from datetime import datetime
from hashlib import sha224

# create an object with review_ID, review_text and the datetime of creation
def create_review(review_text, review_ID):    
    review_obj = {
        "review_ID": review_ID,
        "review_text": review_text,
        "datetime": datetime.now()
    }

    # create a hash of the review_text for creating unique collection with Chromadb
    hash = sha224(str(review_obj).encode()).hexdigest()
    review_obj['hash'] = hash

    return review_obj

# the temporary review_ID
review_ID = 123
# select a comment to test with
review_obj = create_review(review_text, review_ID)

print(review_obj)
print('\n\n')
print('Len of review_text:', len(review_text.split()))

{'review_ID': 123, 'review_text': 'overall good magic card game. the only issue i can really see is the random algorithem for shuffeling the deck, more times than not you either hit a major mana pocket or no mana at all for half the deck reguardless of deck size. i would also suggest to the devs to allow specific card buying for coins as its really hard to get a single card from an entire set of cards, and yes i would expect the cost to be much higher than for booster packs, adjusted for card value and rareity.', 'datetime': datetime.datetime(2024, 4, 4, 11, 32, 51, 626398), 'hash': 'b2327feea64e47779f10da19c0685e8b8e84bae1efd8903c7a9f2434'}



Len of review_text: 94


In [3]:
# define callbacks for detecting token usage
# copied from ../previous_tests/ollama_get_token_usage_example.ipynb

from langchain_core.callbacks.base import BaseCallbackHandler
from langchain_core.outputs.llm_result import LLMResult
from collections import deque

class TokenUsageCallbackHandler(BaseCallbackHandler):

    def __init__(self, deque: deque = None):
        super().__init__()
        self.deque = deque

    def on_llm_end(self, response: LLMResult, **kwargs) -> None:

        generation = response.generations[0][0]
        gen_info = generation.generation_info

        # get token usage
        # ref: https://github.com/orgs/langfuse/discussions/1179
        token_usage = gen_info.get('prompt_eval_count', 0) + gen_info.get('eval_count', 0)
        # get time costed (local machine)
        # instead of getting total duration, we get the prompt_eval_duration and eval_duration to exclude the load duration (e.g. to load the model to the gpu, etc.)
        time_costed = gen_info.get('prompt_eval_duration', 1e-10) + gen_info.get('eval_duration', 1e-10)     # in ns, a small value to indicate a inf time when it fails


        # create an object to store the token usage and time costed
        token_usage_obj = {
            'token_usage': token_usage,
            'time_costed': time_costed
        }

        # append the object to the deque
        self.deque.append(token_usage_obj)



common_deque = deque()
chain_config = {
    "callbacks": [TokenUsageCallbackHandler(common_deque)],
}

In [4]:
import chromadb

# use docker
# chroma_client = chromadb.HttpClient(host="localhost", port=8000)

In [5]:
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

from langchain.text_splitter import CharacterTextSplitter, TokenTextSplitter, RecursiveCharacterTextSplitter
from transformers import AutoTokenizer
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.vectorstores import Chroma

# also change this to use Mistral Embeddings
embedding_func = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

text_spliter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name='cl100k_base', chunk_size=250, chunk_overlap=40
)

docs = text_spliter.create_documents(
    [review_text], metadatas=[{"source":"review_01"}]
)

temp_db = Chroma.from_documents(
    documents=docs,
    embedding=embedding_func,
    collection_name=review_obj['hash']
)

retriever = temp_db.as_retriever(search_kwargs={"k": 5})

/Users/michaelcheng/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/michaelcheng/miniforge3/envs/fyp-test-tm/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


---

prompting

In [6]:
# global constants
GAME_ASPECTS = ['Gameplay', 'Narrative', 'Accessibility', 'Sound', 'Graphics & Art Design', 'Performance', 'Bug', 'Suggestion', 'Price', 'Overall']

3-3-4

with one-shot output format provided

In [7]:
# 3-3-4
# modified from main branch

from langchain_core.prompts import PromptTemplate, ChatPromptTemplate

token_usage_json_list = []
embedding_usage_info_list = []


for (start, end) in [(0, 3), (3, 6), (6, 10)]:
    aspects = GAME_ASPECTS[start:end]

    my_question = _prompts.QUESTION_TEMPLATE_01 + f"{'is ' if len(aspects) <= 1 else 'are '}" + ': ' + f'{aspects}'
    output_format = _prompts.OUTPUT_FORMAT_TEMPATE.format(
        aspects_list_01=str(aspects)[1:-1].replace('\'', '\"'), output_json_template=str({k: '...' for k in aspects}).replace('\'', '\"')
    )

    relevant_docs = retriever.get_relevant_documents(query=my_question, k=5)


    prompt = PromptTemplate(
        template=_prompts.KEYWORD_TEMPLATE_01,
        input_variables=["aspects", 'output_format', 'summaries'],
    )

    chain = prompt | llm

    _resp = chain.invoke({
        "aspects": aspects,
        "output_format": output_format,
        "summaries": str('\n'.join([d.page_content for d in relevant_docs]))
    }, config=chain_config)

    token_usage_json_list.append(common_deque.pop())

    print(_resp); print('-'*20)

Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


KeyboardInterrupt: 

In [ ]:
token_usage_json_list

[{'token_usage': 403, 'time_costed': 2214556000},
 {'token_usage': 334, 'time_costed': 1521923000},
 {'token_usage': 375, 'time_costed': 2092755000}]

3-3-4 without output format provided

In [ ]:
# without output_format

token_usage_json_list = []
embedding_usage_info_list = []


for (start, end) in [(0, 3), (3, 6), (6, 10)]:
    aspects = GAME_ASPECTS[start:end]

    my_question = _prompts.QUESTION_TEMPLATE_01 + f"{'is ' if len(aspects) <= 1 else 'are '}" + ': ' + f'{aspects}'
    output_format = _prompts.OUTPUT_FORMAT_TEMPATE.format(
        aspects_list_01=str(aspects)[1:-1].replace('\'', '\"'), output_json_template=str({k: '...' for k in aspects}).replace('\'', '\"')
    )

    relevant_docs = retriever.get_relevant_documents(query=my_question, k=5)


    prompt = PromptTemplate(
        template=_prompts.KEYWORD_TEMPLATE_01,
        input_variables=["aspects", 'output_format', 'summaries'],
    )

    chain = prompt | llm

    _resp = chain.invoke({
        "aspects": aspects,
        # "output_format": output_format,
        "output_format": '',
        "summaries": str('\n'.join([d.page_content for d in relevant_docs]))
    }, config=chain_config)

    token_usage_json_list.append(common_deque.pop())

    print(_resp); print('-'*20)

Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


 {
"Accessibility": "Suggestions for improving specific card buying and mana distribution in the game."
}
--------------------


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


 {
"Performance": "Issues with random shuffling algorithm leading to inconsistent mana distribution.",
"Graphics & Art Design": "NA",
"Sound": "NA"
}
--------------------
 {
"Suggestion": "i would suggest to the devs to allow specific card buying for coins",
"Bug": "the random algorithm for shuffeling the deck is not working properly, resulting in either having a major mana pocket or no mana at all for half the deck",
"Price": "NA",
"Overall": "overall good magic card game"
}
--------------------


In [ ]:
token_usage_json_list

[{'token_usage': 243, 'time_costed': 926908000},
 {'token_usage': 260, 'time_costed': 1188780000},
 {'token_usage': 306, 'time_costed': 1970716000}]

---

2-2-2-2-2 prompting

with output_format provided

In [ ]:
token_usage_json_list = []
embedding_usage_info_list = []


for (start, end) in [(0, 2), (2, 4), (4, 6), (6, 8), (8, 10)]:
    aspects = GAME_ASPECTS[start:end]

    my_question = _prompts.QUESTION_TEMPLATE_01 + f"{'is ' if len(aspects) <= 1 else 'are '}" + ': ' + f'{aspects}'
    output_format = _prompts.OUTPUT_FORMAT_TEMPATE.format(
        aspects_list_01=str(aspects)[1:-1].replace('\'', '\"'), output_json_template=str({k: '...' for k in aspects}).replace('\'', '\"')
    )

    relevant_docs = retriever.get_relevant_documents(query=my_question, k=5)


    prompt = PromptTemplate(
        template=_prompts.KEYWORD_TEMPLATE_01,
        input_variables=["aspects", 'output_format', 'summaries'],
    )

    chain = prompt | llm

    _resp = chain.invoke({
        "aspects": aspects,
        "output_format": output_format,
        "summaries": str('\n'.join([d.page_content for d in relevant_docs]))
    }, config=chain_config)

    token_usage_json_list.append(common_deque.pop())

    print(_resp); print('-'*20)

Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


 {"Gameplay": "The shuffling algorithm in the game can result in extreme mana availability or scarcity.", "Narrative": "NA"}
--------------------


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


 {"Accessibility": "NA", "Sound": "The review does not provide information about the game's sound aspects."}
--------------------


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


 {"Graphics & Art Design": "No specific mention in the review", "Performance": "The review mentions an issue with the random algorithm for shuffling the deck, suggesting it may not be consistent."}
--------------------


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


 {"Bug": "The random algorithm for shuffeling the deck results in either hitting a major mana pocket or no mana at all for half the deck.", "Suggestion": "Allow specific card buying for coins as it's hard to get a single card from an entire set of cards."}
--------------------
 {"Overall": "The game is overall good with issues in deck shuffling and difficulty in buying specific cards.", "Price": "Users suggest adjusting cost for specific card buying based on rarity."}
--------------------


In [ ]:
token_usage_json_list

[{'token_usage': 299, 'time_costed': 1201809000},
 {'token_usage': 287, 'time_costed': 1076187000},
 {'token_usage': 314, 'time_costed': 1358286000},
 {'token_usage': 331, 'time_costed': 1737717000},
 {'token_usage': 304, 'time_costed': 1352476000}]

2-2-2-2-2 wihtout output format

In [ ]:
token_usage_json_list = []
embedding_usage_info_list = []


for (start, end) in [(0, 2), (2, 4), (4, 6), (6, 8), (8, 10)]:
    aspects = GAME_ASPECTS[start:end]

    my_question = _prompts.QUESTION_TEMPLATE_01 + f"{'is ' if len(aspects) <= 1 else 'are '}" + ': ' + f'{aspects}'
    output_format = _prompts.OUTPUT_FORMAT_TEMPATE.format(
        aspects_list_01=str(aspects)[1:-1].replace('\'', '\"'), output_json_template=str({k: '...' for k in aspects}).replace('\'', '\"')
    )

    relevant_docs = retriever.get_relevant_documents(query=my_question, k=5)


    prompt = PromptTemplate(
        template=_prompts.KEYWORD_TEMPLATE_01,
        input_variables=["aspects", 'output_format', 'summaries'],
    )

    chain = prompt | llm

    _resp = chain.invoke({
        "aspects": aspects,
        "output_format": '',
        "summaries": str('\n'.join([d.page_content for d in relevant_docs]))
    }, config=chain_config)

    token_usage_json_list.append(common_deque.pop())

    print(_resp); print('-'*20)

Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


 {
"Gameplay": "The gameplay is generally good in this magic card game, but there are issues with the random shuffling algorithm leading to inconsistent mana distribution.",
"Narrative": "NA"
}
--------------------


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


 {
"Accessibility": "No specific accessibility issues mentioned in the review.",
"Sound": "The review does not provide information about the sound aspect of the game."
}
--------------------


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


 {
"Performance": "Issues with random shuffling algorithm leading to inconsistent mana distribution.",
"Graphics & Art Design": "NA"
}
--------------------


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


 {
"Bug": "The random algorithm for shuffling the deck results in either hitting a major mana pocket or no mana at all for half the deck.",
"Suggestion": "Allow specific card buying for coins as it's difficult to get a single card from an entire set of cards."
}
--------------------
 {
"Overall": "The game is overall good with issues in deck shuffling algorithm and difficulty in buying specific cards.",
 "Price": "Users suggest adjusting cost for specific card buying, implying they find it difficult and expect higher cost."
}
--------------------


In [ ]:
token_usage_json_list

[{'token_usage': 263, 'time_costed': 1341228000},
 {'token_usage': 250, 'time_costed': 1142515000},
 {'token_usage': 250, 'time_costed': 1058017000},
 {'token_usage': 282, 'time_costed': 1671484000},
 {'token_usage': 267, 'time_costed': 1434526000}]

---

5-5 with output format provided

In [ ]:
token_usage_json_list = []
embedding_usage_info_list = []


for (start, end) in [(0, 5), (5, 10)]:
    aspects = GAME_ASPECTS[start:end]

    my_question = _prompts.QUESTION_TEMPLATE_01 + f"{'is ' if len(aspects) <= 1 else 'are '}" + ': ' + f'{aspects}'
    output_format = _prompts.OUTPUT_FORMAT_TEMPATE.format(
        aspects_list_01=str(aspects)[1:-1].replace('\'', '\"'), output_json_template=str({k: '...' for k in aspects}).replace('\'', '\"')
    )

    relevant_docs = retriever.get_relevant_documents(query=my_question, k=5)


    prompt = PromptTemplate(
        template=_prompts.KEYWORD_TEMPLATE_01,
        input_variables=["aspects", 'output_format', 'summaries'],
    )

    chain = prompt | llm

    _resp = chain.invoke({
        "aspects": aspects,
        "output_format": output_format,
        "summaries": str('\n'.join([d.page_content for d in relevant_docs]))
    }, config=chain_config)

    token_usage_json_list.append(common_deque.pop())

    print(_resp); print('-'*20)

Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


 {"Gameplay": "Player experiences inconsistent mana distribution during games.", "Accessibility": "NA", "Narrative": "NA", "Sound": "NA", "Graphics & Art Design": "NA"}
--------------------
 {"Bug": "The random algorithm for shuffling the deck results in either a major mana pocket or no mana at all for half the deck.", "Suggestion": "Allow specific card buying for coins.", "Price": "NA", "Overall": "It's a good magic card game with some issues in deck shuffling and purchasing cards."}
{"Performance": "NA"}
--------------------


In [ ]:
token_usage_json_list

[{'token_usage': 360, 'time_costed': 1428415000},
 {'token_usage': 395, 'time_costed': 2175031000}]

5-5 without output format provided

In [ ]:
token_usage_json_list = []
embedding_usage_info_list = []


for (start, end) in [(0, 5), (5, 10)]:
    aspects = GAME_ASPECTS[start:end]

    my_question = _prompts.QUESTION_TEMPLATE_01 + f"{'is ' if len(aspects) <= 1 else 'are '}" + ': ' + f'{aspects}'
    output_format = _prompts.OUTPUT_FORMAT_TEMPATE.format(
        aspects_list_01=str(aspects)[1:-1].replace('\'', '\"'), output_json_template=str({k: '...' for k in aspects}).replace('\'', '\"')
    )

    relevant_docs = retriever.get_relevant_documents(query=my_question, k=5)


    prompt = PromptTemplate(
        template=_prompts.KEYWORD_TEMPLATE_01,
        input_variables=["aspects", 'output_format', 'summaries'],
    )

    chain = prompt | llm

    _resp = chain.invoke({
        "aspects": aspects,
        "output_format": '',
        "summaries": str('\n'.join([d.page_content for d in relevant_docs]))
    }, config=chain_config)

    token_usage_json_list.append(common_deque.pop())

    print(_resp); print('-'*20)

Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


 {
"Accessibility": "Suggestions for improvement include allowing specific card buying with coins and adjusting costs accordingly."
}
--------------------
 {
"Overall": "The game is overall good. However, there are issues with the random algorithm for shuffling the deck.",
"Suggestion": "The suggestion to the devs is to allow specific card buying for coins."
}
--------------------


In [ ]:
token_usage_json_list

[{'token_usage': 254, 'time_costed': 1018136000},
 {'token_usage': 278, 'time_costed': 1377259000}]

---

1 aspect per prompt, with output format

In [ ]:
for aspect in GAME_ASPECTS:
    my_question = _prompts.QUESTION_TEMPLATE_01 + f"is {aspect}"
    output_format = _prompts.OUTPUT_FORMAT_TEMPATE.format(
        aspects_list_01=str([aspect])[1:-1].replace('\'', '\"'), output_json_template=str({aspect: '...'}).replace('\'', '\"')
    )

    relevant_docs = retriever.get_relevant_documents(query=my_question, k=5)

    prompt = PromptTemplate(
        template=_prompts.KEYWORD_TEMPLATE_01,
        input_variables=["aspects", 'output_format', 'summaries'],
    )

    chain = prompt | llm

    _resp = chain.invoke({
        "aspects": [aspect],
        "output_format": output_format,
        "summaries": str('\n'.join([d.page_content for d in relevant_docs]))
    }, config=chain_config)

    token_usage_json_list.append(common_deque.pop())

    print(_resp); print('-'*20)

Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


 {"Gameplay": "Some users have reported issues with random shuffling of decks leading to inconsistent mana distribution."}
--------------------


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


 NA. The review does not provide sufficient information about the narrative aspect of the game.
--------------------


...

 {"Bug": "The shuffling algorithm for deck randomization is inconsistent, resulting in extreme mana variations."}
--------------------


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


 {"Suggestion": "The reviews suggest allowing specific card buying for coins and improving the deck shuffling algorithm."}
--------------------


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


 {"Price": "Users suggest adjusting costs for specific card buying to reflect rarity."}
--------------------
 {"Overall": "The game is overall good, but the shuffling algorithm for the deck could use improvement.", "Suggestions": "Allow specific card buying with coins and adjust costs accordingly."}
--------------------


In [ ]:
token_usage_json_list

[{'token_usage': 254, 'time_costed': 1018136000},
 {'token_usage': 278, 'time_costed': 1377259000},
 {'token_usage': 276, 'time_costed': 987202000},
 {'token_usage': 269, 'time_costed': 817757000},
 {'token_usage': 275, 'time_costed': 972698000},
 {'token_usage': 263, 'time_costed': 819742000},
 {'token_usage': 265, 'time_costed': 705706000},
 {'token_usage': 272, 'time_costed': 927306000},
 {'token_usage': 275, 'time_costed': 970371000},
 {'token_usage': 277, 'time_costed': 950976000},
 {'token_usage': 265, 'time_costed': 862515000},
 {'token_usage': 291, 'time_costed': 1288217000}]

1 aspect per prompt, without output format

In [ ]:
for aspect in GAME_ASPECTS:
    my_question = _prompts.QUESTION_TEMPLATE_01 + f"is {aspect}"
    output_format = _prompts.OUTPUT_FORMAT_TEMPATE.format(
        aspects_list_01=str([aspect])[1:-1].replace('\'', '\"'), output_json_template=str({aspect: '...'}).replace('\'', '\"')
    )

    relevant_docs = retriever.get_relevant_documents(query=my_question, k=5)

    prompt = PromptTemplate(
        template=_prompts.KEYWORD_TEMPLATE_01,
        input_variables=["aspects", 'output_format', 'summaries'],
    )

    chain = prompt | llm

    _resp = chain.invoke({
        "aspects": [aspect],
        "output_format": '',
        "summaries": str('\n'.join([d.page_content for d in relevant_docs]))
    }, config=chain_config)

    token_usage_json_list.append(common_deque.pop())

    print(_resp); print('-'*20)

Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


 {
"Gameplay": "Some users have reported issues with the random shuffling algorithm in the game leading to inconsistent mana distribution."
}

{
"Gameplay": "Users suggest allowing specific card buying for coins as it is currently difficult to obtain desired cards from booster packs."
}
--------------------


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


 {
"Narrative": "The reviews mention issues with deck shuffling algorithm leading to inconsistent mana distribution and difficulty in buying specific cards."
}
--------------------


...

 {
"Bug": "The random algorithm for shuffling the deck results in either hitting a major mana pocket or no mana at all for half the deck, regardless of deck size."
}
--------------------


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


 {
"Suggestion": "Players suggest allowing specific card buying for coins and improving the shuffling algorithm in the game."
}
--------------------


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


 {
"Price": "Users suggest adjusting the cost of specific card buying to reflect rarity and difficulty in obtaining cards."
}
--------------------
 {
"Overall": "The game is overall good, but the random shuffling algorithm for the deck can be improved as players often encounter major mana pockets or no mana at all for half the deck regardless of deck size. Additionally, allowing specific card buying for coins would be a suggested improvement as getting a single card from an entire set is difficult."
}
--------------------
